# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

1. Simulating data 1/1
2. run_analysis function 2/2
3. repeat_analysis() function 3/3
4. Testing different sample sizes 2/2
5. Reflection 2/2


---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [ ]:

sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c


simulate_data <- function(sample_size, age_lo, age_hi, beta_xa,
                          beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
age <- runif(sample_size, age_lo, age_hi)
epsilon_x <- rnorm(sample_size, mean = 0, sd = sd_x)
x <- beta_xa * age + beta_x0 + epsilon_x
epsilon_c <- rnorm (sample_size, mean = 0, sd =  sd_c)
c <- beta_ca * age + beta_cx * x + beta_c0 + epsilon_c
data.frame(age = age, x = x, c = c)
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,114.9350,43.09597,263.6979
2,141.6964,76.62873,404.3440
3,191.1964,113.20863,520.8534
4,130.6572,77.58605,359.1422
5,158.3073,74.61072,363.8891
6,152.3144,55.91860,413.1839


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [ ]:
install.packages("mediation")
library(mediation)
library(MASS)

library(mediation)

run_analysis <- function(dat) {

fitM <- lm( x ~ age, data= dat)
fitY <- lm( c ~ x + age, data= dat)
fitMed <- mediate(fitM, fitY, treat="age", mediator="x", boot = TRUE, sims = 1000)

acme <- fitMed$d0
acmep <- fitMed$d0.p
ade <- fitMed$z0
adep <- fitMed$z0.p

return(c(ACME = acme, ACME_p = acmep, ADE = ade, ADE_p = adep))
}

analysis <- run_analysis(dat)
print(analysis)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘checkmate’, ‘Rdpack’, ‘zoo’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘Formula’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘mvtnorm’, ‘sandwich’, ‘lpSolve’, ‘Hmisc’, ‘lme4’


Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0


Running nonparametric bootstrap




     ACME    ACME_p       ADE     ADE_p 
1.8633805 0.0000000 0.6470876 0.0360000 


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [ ]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
                            beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {

  simouts <- matrix(NA, nrow = num_simulations, ncol = 4)
  colnames(simouts) <- c("ACME", "ACME_p", "ADE", "ADE_p")

  for (i in 1:num_simulations) {
    print(paste("Running simulation", i))

    dat <- simulate_data(sample_size, age_lo, age_hi,
                         beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

    analysis_result <- tryCatch({
      run_analysis(dat)
    }, error = function(e) {
      print(paste("Error in simulation", i, ":", e$message))
      return(rep(NA, 4))
    })

    simouts[i, ] <- analysis_result
  }

  if (all(is.na(simouts))) {
    stop("All simulations failed. Check for errors in run_analysis() or mediate().")
  }

  ACME_cov <- mean(simouts[, "ACME_p"] < alpha, na.rm = TRUE)
  ADE_cov <- mean(simouts[, "ADE_p"] < alpha, na.rm = TRUE)

  return(list(ACME_Coverage = ACME_cov, ADE_Coverage = ADE_cov))
}


Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [ ]:
set.seed(123)

coverage_results <- repeat_analysis(
  num_simulations = 10,
  alpha = 0.01,
  sample_size = 100,
  age_lo = 80,
  age_hi = 200,
  beta_xa = 0.5,
  beta_x0 = -5,
  sd_x = 50,
  beta_ca = 0.8,
  beta_cx = 3,
  beta_c0 = 10,
  sd_c = 85
)

print(coverage_results)

[1] "Running simulation 1"


Running nonparametric bootstrap




[1] "Running simulation 2"


Running nonparametric bootstrap




[1] "Running simulation 3"


...

[1] "Running simulation 9"


Running nonparametric bootstrap




[1] "Running simulation 10"


Running nonparametric bootstrap




$ACME_Coverage
[1] 1

$ADE_Coverage
[1] 0.6



---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [ ]:
library(purrr)

sample_sizes <- c(50, 75, 100, 125, 150)

run_for_sample_size <- function(sample_size) {
  coverage_results <- repeat_analysis(
    num_simulations = 10,
    alpha = 0.01,
    sample_size = sample_size,
    age_lo = 80,
    age_hi = 200,
    beta_xa = 0.5,
    beta_x0 = -5,
    sd_x = 50,
    beta_ca = 0.8,
    beta_cx = 3,
    beta_c0 = 10,
    sd_c = 85
  )
  return(coverage_results)
}


coverage_results_list <- map(sample_sizes, run_for_sample_size)

names(coverage_results_list) <- sample_sizes





[1] "Running simulation 1"


Running nonparametric bootstrap




[1] "Running simulation 2"


Running nonparametric bootstrap




[1] "Running simulation 3"


...

[1] "Running simulation 9"


Running nonparametric bootstrap




[1] "Running simulation 10"


Running nonparametric bootstrap




$`50`
$`50`$ACME_Coverage
[1] 0.3

$`50`$ADE_Coverage
[1] 0.3


$`75`
$`75`$ACME_Coverage
[1] 0.5

$`75`$ADE_Coverage
[1] 0.5


$`100`
$`100`$ACME_Coverage
[1] 0.7

$`100`$ADE_Coverage
[1] 0.8


$`125`
$`125`$ACME_Coverage
[1] 0.9

$`125`$ADE_Coverage
[1] 0.7


$`150`
$`150`$ACME_Coverage
[1] 0.9

$`150`$ADE_Coverage
[1] 0.8




Print your results.

In [ ]:
print(coverage_results_list)


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> Considering our ACME resullts, I can conclude that a small sample is not going to be enough to detect the mediation effect. For example with 50 participants, we have just 40% coverage. However, by increasing the sample up to 125-150, is more likely to detect this indirect effect with 90% coverage.
> In the case of ADE, we also need more participants, because a sample of 50 will lead to 50% of direct effect. And it is until 150 particpants that reaches 90% coverage.

Overall, if we have recruiting issues a sample size of 100 would be a middleground but it won't give us 90% as originally intended.

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *This can be explained by the fact our direct effect was associated with the mediator. Therefore, it shrinks and makes it difficult to detect.

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** I used ChatGPT to help me fix the code (It was giving me NA)
> *Someone's Name*